In [369]:
import pymorphy2
from multiprocessing.dummy import Pool, Queue
import lxml
from tqdm import tqdm
from time import sleep
from bs4 import BeautifulSoup
from bs4.element import Comment
import gzip
import pandas as pd
import codecs
import sys
import os
import json
import re
from nltk.stem.snowball import SnowballStemmer

In [365]:
regex_symbols = re.compile(r'[\n\r\t|()#№^«»@!_&/><?%\[\]©·•]')

In [392]:
my_position_start = 1
my_position_end = 10
queue = Queue() # очередь ссылок на книги
for i in range(my_position_start, my_position_end):
    queue.put(i)

In [391]:
def process_doc(number_of_doc):
    with open('./content/{:d}.dat'.format(number_of_doc), encoding='utf-8') as file:
    #with open('./content/1.dat', encoding='utf-8') as file:
        print(number_of_doc)
        text_checker = {}
        html_test = file.read()
        soup = BeautifulSoup(html_test, 'html.parser')
        title_of_doc = soup.text[:soup.text.index('\n')]
        text = soup.text[soup.text.index('\n'):]
        text = regex_symbols.sub(" ", text)
        text = re.sub('title_of_doc', '', text)
        text = re.sub("\s\s+" , " ", text)
        text_checker[number_of_doc] = (title_of_doc, text) 
        text_checker[title_of_doc] = text
        return text_checker

In [393]:
def process_all_docs(i):
    with gzip.open('data/part_{:05d}.jsonl.gz'.format(i), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)

        while not queue.empty():
            try:
                id_new = queue.get()
                record = process_doc(id_new)
            except Exception as e:
                print(id_new, file=sys.stderr)
                print(e, file=sys.stderr)
            record_str = json.dumps(record, ensure_ascii=False)
            print(record_str, file=f_json)

            # счетчик должен атомарно обновиться
            with lock:
                pbar.update(1)


with Pool(processes=8) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_all_docs, range(pool._processes))

  0%|          | 0/9 [00:00<?, ?it/s]

1
34

2
75
6



 22%|██▏       | 2/9 [00:00<00:01,  4.35it/s]

8
9


100%|██████████| 9/9 [00:01<00:00,  8.56it/s]
